- Simplified self-attention, without trainable weights
- Self-attention witn trainable weights
- Causal attention
- Multi-head attention

In [2]:
import torch

**a) Simplified self-attention (without trainabel weights)**

In [25]:
#Example. input sentence embedded into 3-dimensional vectors.
inputs = torch.tensor(
[[0.43, 0.15, 0.89], # Your (x^1)
[0.55, 0.87, 0.66], # journey (x^2)
[0.57, 0.85, 0.64], # starts (x^3)
[0.22, 0.58, 0.33], # with (x^4)
[0.77, 0.25, 0.10], # one (x^5)
[0.05, 0.80, 0.55]] # step (x^6)
)


Step 1. Calculate attention scores. w21 is the attention xscore between x^1 and the emedded query token q2, 
#which in this cases is x^2
- q2=x2
- w21=x1q2
- w22=x2q2
- w23=x3q2
- w24=x4q2
- w25=x5q2
- w26=x6q2

In [51]:
query=inputs[1]
attn_scores_2=torch.empty(inputs.shape[0]) #to store attention scores for x2
for i,xi in enumerate(inputs):
    attn_scores_2[i]=torch.dot(xi,query)
print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


Step 2. Normalize the attention scores

In [27]:
#Normalizing using softmax (better at handling extreme values and has more desirable gradient properties during training)
attn_weights_2= torch.softmax(attn_weights_2,dim=0)
print(attn_weights_2_norm)

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])


Step 3. Compute the context vector z2 by multiplying the embedded input tokens xi with the attention weights and sum the resulting vectors.

In [46]:
query=inputs[1]
context_vec_2=torch.zeros(query.shape) #to store attention scores for x2
for i,xi in enumerate(inputs):
    context_vec_2+=attn_weights_2[i]*xi
print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


In [47]:
#Computing for all input tokens
attn_scores=torch.empty(inputs.shape[0],inputs.shape[0])
for i, xi in enumerate(inputs):
    for j,xj in enumerate(inputs):
            attn_scores[i,j]=torch.dot(xi,xj)
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [48]:
#Faster way 
attn_scores=inputs@inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [50]:
#Normalizing
attn_weights=torch.softmax(attn_scores,dim=1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [52]:
#Context vector
all_context_vecs=attn_weights@inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


**b) Self-attention with trainable weights (scaled dot-product attention)**

Difference with the simplified model before is the introduction of trainable weight matrices that are updated during model training. There are three weigh matrices Wq, Wk, and Wv used to project the embedded input tokens xi into query, key, and value vectors.

In [ ]:
x2=inputs[1]
d_in=inputs.shape[1]
d_out=2 #the input and output dimensions are usually the same, but for illustration purposes and easier computation selected 2